### Import Required Libraries

In [6]:
import numpy as np
from scipy.integrate import odeint
from scipy.signal import butter, filtfilt, find_peaks
from scipy.stats import gaussian_kde,ecdf
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy.optimize import fsolve
import time
import pandas as pd
import os

### Viscous_parameters_solver Function

In [7]:
def viscous_parameters_solver(t, theta,mobject):
    l = 0.6505
    r = 0.36115625
    g = 9.7949
    mnet = 2.974+mobject #FIRST NUMBER IS DISK AND SENSOR, NOT INCLUDING TRANSMITTER
    noise_param = 0.04
    # Filter the data
    b, a = butter(4, 0.05, 'low')
    filtered_theta = filtfilt(b, a, theta)
    peak_filtering_param = 3 #remember to adjust as to remove outliers associated with signal noise

    # Use find_peaks with adjusted parameters to identify peaks
    peaks, _ = find_peaks(filtered_theta, prominence=np.deg2rad(noise_param*peak_filtering_param))  # adjust prominence if needed

    # Check if peaks are found
    if len(peaks) == 0:
        print("No peaks were found.")
        return [0, 0, 0, 0]

    # Calculate the periods between peaks
    periods = np.diff(t[peaks])
    # plt.figure()
    # plt.plot(t,theta,'kx',ms=1)
    # plt.plot(t,filtered_theta,'r-')
    # plt.xlim((0,15))
    # plt.show()
    # plt.figure()
    # plt.plot(periods)
    # plt.show()
    # Calculate median filtered period
    T_d = np.median(periods)
    # print(f"T_d = {T_d}", end='\n')
    omega_d = (2 * np.pi) / T_d
    # print(f"Calculated wd = {omega_d}", end='\n')

    # Calculate the damping ratio
    amplitude_ratios = filtered_theta[peaks][1:] / filtered_theta[peaks][:-1]
    damping_ratios = -1*np.log(amplitude_ratios)
    # print(f"Calculated dr = {np.median(damping_ratios)}",end='\n')
    zeta = np.median(damping_ratios) / ( np.sqrt( (4 * np.pi ** 2) + (np.median(damping_ratios) ** 2) ) )
    # print(f"Calculated zeta = {zeta}",end='\n')

    # Calculate natural frequency and inertia
    omega_n = omega_d / np.sqrt(1 - zeta ** 2)
    Inet_pure = (mnet * g * r ** 2) / (l * omega_n ** 2)
    c_pure = zeta * 2 * np.sqrt(Inet_pure * ((mnet * g * r ** 2) / l))

    return [Inet_pure, (mnet * g * r ** 2) / l, c_pure, 0]


### Import Data


In [8]:
folder_path = '/home/coder/workspace/Finnamore/Mass_Calibration'

# Numer of Trials per Test
numTests = 45
raw_data = {}
# Define the pattern or names of your CSV files
for i in range(numTests):
    raw_data[i] = {}
    file_name_disk = 'I_D_{:02}.csv'.format(i)
    file_path_disk = os.path.join(folder_path, file_name_disk)
    file_name_full = 'I_.4465_{:02}.csv'.format(i)
    file_path_full = os.path.join(folder_path, file_name_full)
    df_disk = pd.read_csv(file_path_disk, usecols=[0, 1])
    df_full = pd.read_csv(file_path_full, usecols=[0, 1])
    raw_data[i]['Disk'] = df_disk.values
    raw_data[i]['Full'] = df_full.values

# Converting ms to s
for i in range(numTests):
    raw_data[i]['Disk'][:,0] = np.divide(raw_data[i]['Disk'][:,0],1000)
    raw_data[i]['Full'][:,0] = np.divide(raw_data[i]['Full'][:,0],1000)


### Calculate Inertial Values

In [11]:
inertia_values = np.zeros((numTests,4))
types = ['Disk','Full']
# I_actual in this case is a towel roll
I_object = 0.5*.9865*((.15533/2)**2+(.03723/2)**2)

for i in range(numTests):
    inertia_values[i][0] = viscous_parameters_solver(raw_data[i]['Disk'][:,0], raw_data[i]['Disk'][:,1],0)[0]
    inertia_values[i][1] = viscous_parameters_solver(raw_data[i]['Full'][:,0], raw_data[i]['Full'][:,1],.9865)[0]
    inertia_values[i][2] = inertia_values[i][1]-inertia_values[i][0]
    inertia_values[i][3] = ((inertia_values[i][2]-I_object)/(I_object))*100


No peaks were found.
No peaks were found.
No peaks were found.
No peaks were found.
No peaks were found.
No peaks were found.
No peaks were found.
No peaks were found.
No peaks were found.
No peaks were found.
No peaks were found.
No peaks were found.
No peaks were found.
No peaks were found.
No peaks were found.
No peaks were found.
No peaks were found.
No peaks were found.
No peaks were found.
No peaks were found.
No peaks were found.
No peaks were found.
No peaks were found.
No peaks were found.
No peaks were found.
No peaks were found.


/tmp/ipykernel_96712/2554623644.py:38: RuntimeWarning: invalid value encountered in log
  damping_ratios = -1*np.log(amplitude_ratios)


### Plot/Print Results

In [10]:
# KDE plot for Results
# REMEMBER TO CHECK PARAMS INSIDE VISCOUS_SOLVER (ESPECIALLY MNET)

I_disk_mean = np.median(inertia_values[:,0])
I_full_mean = np.median(inertia_values[:,1])
I_object_mean = np.median(inertia_values[:,2])
I_error_mean = np.median(inertia_values[:,3])

# Gaussian Distributions
kde_I_disk = gaussian_kde(inertia_values[:,0])
kde_I_full = gaussian_kde(inertia_values[:,1])
kde_I_object = gaussian_kde(inertia_values[:,2])
kde_I_error = gaussian_kde(inertia_values[:,3])

xi_disk = np.linspace(np.min(inertia_values[:,0]), np.max(inertia_values[:,0]), 1000)
xi_full = np.linspace(np.min(inertia_values[:,1]), np.max(inertia_values[:,1]), 1000)
xi_object = np.linspace(np.min(inertia_values[:,2]), np.max(inertia_values[:,2]), 1000)
xi_error = np.linspace(np.min(inertia_values[:,3]), np.max(inertia_values[:,3]), 1000)
f = kde_I_error(xi_error)

# Disk and Full
plt.figure()
plt.plot(xi_disk, kde_I_disk(xi_disk),color='blue',ls='-')
plt.fill_between(xi_disk,np.zeros(1000),kde_I_disk(xi_disk),color='blue',alpha=0.2)
plt.plot(xi_full, kde_I_full(xi_full), color='red',ls='-')
plt.fill_between(xi_full,np.zeros(1000),kde_I_full(xi_full),color='red',alpha=0.2)
plt.xlim(np.min(inertia_values[:,0]), np.max(inertia_values[:,1]))
plt.ylim([0,np.max(kde_I_full(xi_full))*1.1])
plt.axvline(x=I_disk_mean, color='blue', label='Disk Inertia',ls='--')
plt.axvline(x=I_full_mean, color='red', label='Full Setup Inertia',ls='--')
plt.xlabel('Measured Moment of Inertia')
plt.ylabel('Density Estimate')
plt.title('Measured Moments of Inertia')
plt.legend()
plt.show()

# Object
plt.figure()
plt.plot(xi_object, kde_I_object(xi_object), color='black',ls='-')
plt.fill_between(xi_object,np.zeros(1000),kde_I_object(xi_object),color='black',alpha=0.2)
plt.xlabel('Measured Moment of Inertia')
plt.ylabel('Density Estimate')
plt.title('Measured Moments of Inertia')
plt.xlim([np.min(inertia_values[:,2]), np.max(inertia_values[:,2])])
plt.ylim([0,np.max(kde_I_object(xi_object))*1.1])
plt.axvline(x=I_object_mean, color='black', label='Measured Object Inertia',ls='--')
plt.axvline(x=I_object, color='red', label='Actual Object Inertia',ls='-')
plt.legend()
plt.show()

# Percent Error
plt.figure()
plt.plot(xi_error, kde_I_error(xi_error), color='black',ls='-')
plt.fill_between(xi_error,np.zeros(1000),kde_I_error(xi_error),color='black',alpha=0.2)
plt.xlabel('Percent Error')
plt.ylabel('Density Estimate')
plt.title('Percent Error')
plt.xlim([-50,np.max(inertia_values[:][3])])
plt.ylim([0,np.max(kde_I_error(xi_error))*1.1])
plt.axvline(x=I_error_mean, color='black', label='Mean Error',ls='--')
plt.axvline(x=0, color='red', label='Error = 0',ls='-')
plt.legend()
plt.show()

ValueError: array must not contain infs or NaNs